In [ ]:
import pandas as pd
import json
import datetime
import time
import os
import numpy as np
from config import DATA_METADATA_PROCESSED_FILE, DATA_BIM_RAW_FOLDER, DATA_SOLAR_GAINS_PROCESSED_FILE
import matplotlib

In [ ]:
meatadata =  pd.read_excel(DATA_METADATA_PROCESSED_FILE, sheets='SENSORS')
output_path = DATA_SOLAR_GAINS_PROCESSED_FILE
meatadata

In [ ]:
buildings = list(set(meatadata['ID_CEA'].values))
scenarios = {'2018': pd.date_range(start='2018-01-01', end='2018-12-31 23:00:00', freq='H'),
             '2019': pd.date_range(start='2019-01-01', end='2019-12-31 23:00:00', freq='H')}
df = pd.DataFrame()
for building in buildings:
    for scenario, timestamp in scenarios.items():
        sg_windows_path = os.path.join(DATA_BIM_RAW_FOLDER, scenario, 'outputs\data\solar-radiation', building+ "_radiation.csv")
        sg_path = os.path.join(DATA_BIM_RAW_FOLDER, scenario, 'outputs\data\demand', building+ ".csv")
        solar_gains_windows_area = pd.read_csv(sg_windows_path)[['windows_west_m2','windows_east_m2', 'windows_south_m2', 'windows_north_m2']].sum(axis=1)[0]
        solar_gains_df = pd.read_csv(sg_path)[['I_sol_kWh', 'Name']]
        solar_gains_df['SG_Whm2'] = solar_gains_df['I_sol_kWh']/solar_gains_windows_area
        solar_gains_df = solar_gains_df.set_index(timestamp)
        solar_gains_df = solar_gains_df.resample('D').sum()
        solar_gains_df['ID_CEA'] = building
        solar_gains_df['timestamp'] = solar_gains_df.index
        df = df.append(solar_gains_df, ignore_index=True)
df
#get information about the sola radiation of buildings.

In [ ]:
df.to_csv(output_path, index=False)

In [ ]:
df = df.set_index(df['timestamp'])
visual = df[df['ID_CEA'] == buildings[10]]
ax = visual['SG_Whm2'].plot(color = ['C8'])
font = {'family' : 'Arial',
        'size'   : 18}
ax.set_xlabel("")
ax.set_ylabel("Daily Solar Gains [kWh/m2]")
ax.legend(["Solar Gains"]);
matplotlib.rc('font', **font)